In [1]:

from keys import FTX_API_KEY, FTX_API_SECRET
from datetime import datetime
import time
import hmac
import pandas as pd
from requests import Request, Session, Response
from typing import Optional, Dict, Any, List


class FtxClient:


    def __init__(self, api_key=None, api_secret=None, subaccount_name=None):
            self._session = Session()
            self._api_key = api_key
            self._api_secret = api_secret
            self._subaccount_name = subaccount_name
            self._ENDPOINT = 'https://ftx.com/api/'

    def _get(self, path: str, params: Optional[Dict[str, Any]] = None) -> Any:
            return self.send_request('GET', path, params=params)

    def sign_request(self, request: Request):
        ts = int(time.time() * 1000)
        request = Request('GET', self._ENDPOINT)
        prepared = request.prepare()
        print(prepared)
        signature_payload = f'{ts}{prepared.method}{prepared.path_url}'.encode()
        signature = hmac.new(self._api_secret.encode(), signature_payload, 'sha256').hexdigest()

        request.headers['FTX-KEY'] = self._api_key
        request.headers['FTX-SIGN'] = signature
        request.headers['FTX-TS'] = str(ts)

    def send_request(self, method: str, path: str, **kwargs):
        request = Request(method, self._ENDPOINT + path, **kwargs)
        self.sign_request(request)
        response = self._session.send(request.prepare())
        return self._process_response(response)

    def _process_response(self, response: Response):
        try:
            data = response.json()
        except ValueError:
            response.raise_for_status()
            raise
        else:
            if not data['success']:
                raise Exception(data['error'])
            return data['result']
    
    def _get_all_trades(self, market: str, start_time: float = None, end_time: float = None):
        ids = set()
        limit = 100
        results = []
        while True:
            response = self._get(f'markets/{market}/trades', {
                'end_time': end_time,
                'start_time': start_time,
            })
            deduped_trades = [r for r in response if r['id'] not in ids]
            results.extend(deduped_trades)
            ids |= {r['id'] for r in deduped_trades}
            print(f'Adding {len(response)} trades with end time {end_time}')
            if len(response) == 0:
                break
            end_time = min(parse_datetime(t['time']) for t in response).timestamp()
            if len(response) < limit:
                break
        return results


## getting all futures tickers

In [6]:
import json

# dt = datetime(2021, 1, 1).timestamp()
acc = FtxClient(api_key=FTX_API_KEY, api_secret=FTX_API_SECRET)
current_futures = acc._get('futures')
for ticker in a: 
    print(ticker['name'])

# res = acc.send_request('GET', 'markets/BTC-PERP/candles?resolution=15&limit=10')
# all_trade = acc._get_all_trades('BTC-PERP', start_time='1559881511', end_time='1609477200')
# df = pd.DataFrame(res)
# print(df)

<PreparedRequest [GET]>
1INCH-PERP
1INCH-0326
1INCH-0625
AAPL-0326
AAPL-0625
AAVE-PERP
AAVE-0326
AAVE-0625
ABNB-0326
ABNB-0625
ACB-0326
ACB-0625
ADA-PERP
ADA-0326
ADA-0625
ALGO-PERP
ALGO-0326
ALGO-0625
ALPHA-PERP
ALT-PERP
ALT-0326
ALT-0625
AMC-0326
AMC-0625
AMD-0326
AMD-0625
AMPL-PERP
AMZN-0326
AMZN-0625
APHA-0326
APHA-0625
AR-PERP
ARKK-0326
ARKK-0625
ATOM-PERP
ATOM-0326
ATOM-0625
AVAX-PERP
AVAX-0326
AVAX-0625
BABA-0326
BABA-0625
BADGER-PERP
BAL-PERP
BAL-0326
BAL-0625
BAND-PERP
BAO-PERP
BAT-PERP
BB-0326
BB-0625
BCH-PERP
BCH-0326
BCH-0625
BILI-0326
BILI-0625
BITW-0326
BITW-0625
BNB-PERP
BNB-0326
BNB-0625
BNT-PERP
BNTX-0326
BNTX-0625
BRZ-PERP
BRZ-0326
BRZ-0625
BSV-PERP
BSV-0326
BSV-0625
BTC-PERP
BTC-MOVE-0320
BTC-MOVE-0321
BTC-0326
BTC-MOVE-2021Q1
BTC-MOVE-WK-0326
BTC-MOVE-WK-0402
BTC-MOVE-WK-0409
BTC-MOVE-WK-0416
BTC-0625
BTC-MOVE-2021Q2
BTC-0924
BTC-MOVE-2021Q3
BTC-1231
BTC-HASH-2021Q1
BTMX-PERP
BTMX-0326
BTMX-0625
BTT-PERP
BYND-0326
BYND-0625
CAKE-PERP
CEL-0625
CGC-0326
CGC-0625
CHZ-P

## getting all expiered futures

In [11]:
expired_futures = acc._get('expired_futures')
for ticker in expired_futures: 
    print(ticker['name'], ticker['expiryDescription'])

cember 2020
ARKK-1225 December 2020
EOS-1225 December 2020
BTMX-1225 December 2020
LINK-1225 December 2020
SOL-1225 December 2020
XAUT-1225 December 2020
MID-1225 December 2020
BSV-1225 December 2020
DOT-1225 December 2020
DMG-1225 December 2020
BCH-1225 December 2020
SUSHI-1225 December 2020
TWTR-1225 December 2020
BNB-1225 December 2020
TLRY-1225 December 2020
SPY-1225 December 2020
BABA-1225 December 2020
ATOM-1225 December 2020
MATIC-1225 December 2020
OKB-1225 December 2020
BAL-1225 December 2020
GOOGL-1225 December 2020
BTC-HASH-2020Q4 2020 Q4
SQ-1225 December 2020
TOMO-1225 December 2020
MRNA-1225 December 2020
RUNE-1225 December 2020
ABNB-1225 December 2020
CREAM-1225 December 2020
SHIT-1225 December 2020
GRT-1225 December 2020
THETA-1225 December 2020
HNT-1225 December 2020
LTC-1225 December 2020
FIL-1225 December 2020
BTC-MOVE-1224 2020-12-24
BTC-MOVE-1223 2020-12-23
BTC-MOVE-1222 2020-12-22
BTC-MOVE-1221 2020-12-21
BTC-MOVE-1220 2020-12-20
BTC-MOVE-1219 2020-12-19
BTC-MOVE-W

## getting specific expired futures data 

In [19]:
eth_hist_futures = acc._get('markets/ETH-0925/candles?resolution=60&start_time=1596254400&end_time=1598932800')
print(eth_hist_futures)

me': '2020-09-01T01:48:00+00:00', 'time': 1598924880000.0, 'open': 440.28, 'high': 441.2, 'low': 440.22, 'close': 441.2, 'volume': 8002.0444}, {'startTime': '2020-09-01T01:49:00+00:00', 'time': 1598924940000.0, 'open': 441.2, 'high': 441.34, 'low': 441.02, 'close': 441.2, 'volume': 0.0}, {'startTime': '2020-09-01T01:50:00+00:00', 'time': 1598925000000.0, 'open': 441.2, 'high': 441.2, 'low': 440.59, 'close': 440.59, 'volume': 0.0}, {'startTime': '2020-09-01T01:51:00+00:00', 'time': 1598925060000.0, 'open': 440.59, 'high': 441.01, 'low': 440.59, 'close': 440.79, 'volume': 6611.85}, {'startTime': '2020-09-01T01:52:00+00:00', 'time': 1598925120000.0, 'open': 440.79, 'high': 440.79, 'low': 440.37, 'close': 440.71, 'volume': 0.0}, {'startTime': '2020-09-01T01:53:00+00:00', 'time': 1598925180000.0, 'open': 440.71, 'high': 440.71, 'low': 440.12, 'close': 440.12, 'volume': 0.0}, {'startTime': '2020-09-01T01:54:00+00:00', 'time': 1598925240000.0, 'open': 440.12, 'high': 440.26, 'low': 439.77, 'c

2021-03-07 00:00:01
